
#  Preguntas

###  Modificamos datos y cargamos librerias  

In [ ]:
import ETL
clientes = ETL.clientes
alquileres = ETL.alquileres
peliculas = ETL.peliculas

import pandas as pd
from datetime import timedelta
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
# Precio alquiler
PRECIO_ALQUILER = 3

# Formato fecha
alquileres["fecha_alquiler"] = pd.to_datetime(alquileres["fecha_alquiler"])

# Agrupa por periodos año-mes 
alquileres["mes"] = alquileres["fecha_alquiler"].dt.to_period("M")


# Tabla de películas unica por movie_id 
peliculas_unicas = peliculas.drop_duplicates(subset="movie_id")[["movie_id", "title"]]

clientes_reducido = clientes[["cliente_id", "nombre", "apellido"]]

In [ ]:
!pip install ipywidgets

In [ ]:
!pip install jupyterlab_widgets

##  Muestra desplegable para poder elegir el cliente y mostras las peliculas alquiladas

In [ ]:

salida = widgets.Output()
# Creo el desplegable
desplegable = widgets.Dropdown(
    options=clientes["nombre"].tolist(),
    description="Cliente:"
)
# Mostramos el despegable con los nombres
display(desplegable, salida)

#Se ejecuta cuando el .observe cambia
def cuando_cambia_cliente(cambio):
    if cambio["name"] != "value":
        return

    with salida:
        #Limpia la salida
        salida.clear_output()
        # Seleccionamos un nuevo cliente
        nombre_sel = cambio["new"]
        # Cogemos el id
        cliente_id_sel = clientes.loc[clientes["nombre"] == nombre_sel, "cliente_id"].iloc[0]
        #Filtramos los alquileres
        alq_cliente = (
            alquileres[alquileres["cliente_id"] == cliente_id_sel]
            .sort_values("fecha_alquiler", ascending=False)
        )
        
       #Obtenemos el titulo por el id
        tabla = alq_cliente.merge(
            peliculas_unicas[["movie_id", "title"]],
            on="movie_id",
            how="left"
        )
        #Creamos la tabla final con los datos que nos interesa
        tabla_final = tabla[["fecha_alquiler", "title"]]
        
        ingresos = len(tabla_final) * 3

        
        print(f"Nº de alquileres: {len(tabla_final)}")
        print(f"Ingresos totales: {ingresos} €")
        
        display(tabla_final.head(250))
        
# Conectamos el desplegable
desplegable.observe(cuando_cambia_cliente, names="value")

# Se ejecuta la funcion
cuando_cambia_cliente({"name": "value", "new": desplegable.value})



# Muestra desplegable para poder elegir el periodo de tiempo y mostras las clientes que mas han alquilado 

In [ ]:
salida = widgets.Output()

periodos = {
    "Últimos 7 días": 7,
    "Últimos 30 días": 30,
    "Últimos 365 días": 365,
    "Todo": None
}

desplegable = widgets.Dropdown(
    options=list(periodos.keys()),
    description="Elige un periodo:"
)
display(desplegable, salida)

def cuando_cambia_periodo(cambio):

    if cambio["name"] != "value":
        return

    with salida:
        salida.clear_output()

        texto = cambio["new"]
        dias = periodos[texto]

        # Creamos copia y convertimos SOLO dentro de la función
        alq = alquileres.copy()
        

        if dias is None:
            alq_filtrados = alq
        else:
            fecha_referencia = alq["fecha_alquiler"].max()
            fecha_limite = fecha_referencia - timedelta(days=dias)
            alq_filtrados = alq[alq["fecha_alquiler"] >= fecha_limite]

        ingresos = len(alq_filtrados) * PRECIO_ALQUILER

        print(f"Periodo: {texto}")
        print(f"Alquileres totales: {len(alq_filtrados)}")
        print(f"Lo que es lo mismo: {ingresos} € en ese Periodo")

        resumen = (
            alq_filtrados
            .groupby("cliente_id")
            .size()
            .reset_index(name="num_alquileres")
        )

        resumen = resumen.merge(
            clientes[["cliente_id", "nombre"]],
            on="cliente_id"
        )

        resumen = resumen.sort_values("num_alquileres", ascending=False)

        display(resumen[["nombre", "num_alquileres"]].head(10))

desplegable.observe(cuando_cambia_periodo, names="value")

# Muestra desplegable para poder elegir el pelicula y mostrar el historial de clientes que ha tenido esa pelicula

In [ ]:
# Widget de salida
salida_peli = widgets.Output()

# Selector de película
desplegable_peli = widgets.Dropdown(
    options=peliculas["title"].tolist(),
    description="Película:"
)

display(desplegable_peli, salida_peli)

# Función para cuando se elige una película
def cuando_cambia_peli(cambio):
    if cambio["name"] != "value":
        return
    
    with salida_peli:
        salida_peli.clear_output()
        
        # Título elegido
        titulo_sel = cambio["new"]
        
        # Obtener movie_id
        movie_id_sel = peliculas.loc[peliculas["title"] == titulo_sel, "movie_id"].iloc[0]
        
        # Filtrar alquileres de esa película
        alq_peli = alquileres[alquileres["movie_id"] == movie_id_sel].copy()
        alq_peli = alq_peli.sort_values("fecha_alquiler", ascending=False)
        
        # Añadir nombre del cliente
        tabla = alq_peli.merge(clientes_reducido[["cliente_id", "nombre", "apellido"]], on="cliente_id", how="left")
        
        # Seleccionar columnas a mostrar
        tabla_final = tabla[["fecha_alquiler", "nombre", "apellido"]]
        
        print(f"Pelicula: {titulo_sel}")
        print(f"Numero de alquileres: {len(tabla_final)}")
        display(tabla_final.head(200))

# Conectar evento
desplegable_peli.observe(cuando_cambia_peli, names="value")

# Ejecutar por primera vez
cuando_cambia_peli({"name": "value", "new": desplegable_peli.value})


# Muestra el top de peliculas elegida en una grafica de barras ( se puede modificar el top )

In [ ]:
top_deseado=10

alquileres_por_pelicula = (
    alquileres.groupby("movie_id").size().reset_index(name="num_alquileres")
)

top_peliculas = alquileres_por_pelicula.sort_values( by ="num_alquileres", ascending=False
)
top_peliculas_con_titulo = top_peliculas.merge( peliculas_unicas, on="movie_id"   
)

top=(top_peliculas_con_titulo[["title", "num_alquileres"]].head(top_deseado).dropna(subset=["title"]))

plt.barh(data=top, y="title", width="num_alquileres")

plt.xlabel("Número de alquileres")
plt.ylabel("Película")
plt.title(f"Top {top_deseado} películas más alquiladas")



# Muestra la evolucion de los ingresos desde el primer dia hasta "ahora"

In [ ]:
ingresos_por_mes = (
    alquileres
    .groupby("mes")
    .size()
    .reset_index(name="num_alquileres")
)

# Ingresos 
ingresos_por_mes["ingresos_estimados"] = (
    ingresos_por_mes["num_alquileres"] * PRECIO_ALQUILER
)

# Convertir a fecha
ingresos_por_mes["mes"] = ingresos_por_mes["mes"].dt.to_timestamp()

plt.figure(figsize=(10, 6))
plt.plot("mes", "ingresos_estimados", data=ingresos_por_mes)

plt.xlabel("Mes")
plt.ylabel("Ingresos estimados (€)")
plt.title("Evolución de ingresos")
plt.grid(True, alpha=0.3)
plt.tight_layout()

# Muestra una tabla de las peliculas por generios mas alquiladas 

### Se puede cambiar en la variable top_generos de cuantas pelis quieres ver el top

In [ ]:
top_generos = 3
alquileres_por_pelicula = (alquileres
    .groupby("movie_id")
    .size()
    .reset_index(name="num_alquileres")
)

peliculas_unicas = (peliculas
    .drop_duplicates(subset="movie_id")[["movie_id", "title", "genre_name"]]
)

df_generos = alquileres_por_pelicula.merge(peliculas_unicas
)

alquileres_por_genero_pelicula = ( df_generos
    .groupby(["genre_name", "title"], as_index=False)["num_alquileres"]
    .sum()
)

top_por_genero = ( alquileres_por_genero_pelicula
    .sort_values(["genre_name", "num_alquileres"], ascending=[True, False])
    .groupby("genre_name")
    .head(top_generos)
)
print (f"Se muestra el top {top_generos} de peliculas alquiladas por generos ")
display(top_por_genero)


# Comprobar la relación entre la popularidad de una película en TMDB y el alquiler de ella misma 

In [ ]:
alquileres_por_pelicula = (alquileres
    .groupby("movie_id")
    .size()
    .reset_index(name="num_alquileres")
)

# Películas únicas 
peliculas_unicas = peliculas.drop_duplicates(subset="movie_id")[["movie_id", "title", "popularity"]]

df_popularidad = alquileres_por_pelicula.merge(
    peliculas_unicas,
    on="movie_id",
    how="inner"
)


plt.scatter("popularity", "num_alquileres", data=df_popularidad)

plt.xlabel("Popularidad en TMDB")
plt.ylabel("Número de alquileres")
plt.title("Relación entre popularidad en TMDB y número de alquileres")
